In [2]:
import pandas as pd
import requests
import numpy as np
from io import StringIO
import time
from datetime import datetime, timedelta
import calendar
import datetime as dt
from functools import reduce
import json
import os
from collections import Counter
from pathlib import Path, PureWindowsPath

In [ ]:
class Price():
    def __init__(self, year, month, day):
        url = f'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date={year}{month:02}{day:02}&type=ALL'

        self.year = year
        self.month = month
        self.day = day
        self.url = url
        
    def get(self):
        r = requests.get(url)
        if r.text=='':
            return False    

        df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                    header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
        df.drop(['Unnamed: 16','最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量','漲跌(+/-)'], axis=1, inplace=True)
        for i in ['成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '本益比']:
            df[i] = df[i].apply(lambda x: pd.to_numeric(x.replace(",", ""), errors='coerce') if type(x)!=float else x)
        df['股價日期'] = datetime(int(year),int(month),int(day),0,0,0)
        self.df = df
        return True
    
    def check_exist(self, dst):
        dst = os.path.join(dst, str(self.year))
        self.fname = os.path.join(dst, str(self.year), f'{self.year}_{self.month}_{self.day}.pkl')
        if not os.path.exists(self.fname):
            return False
        return True
    
    def save(self):
        self.df.to_pickle(self.fname)

In [20]:
def siiPrice(year, month, day, corporation_only=True):
    url = f'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date={year}{month:02}{day:02}&type=ALL'
    r = requests.get(url)
    if r.text=='':
        return None    
    
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    df.drop(['Unnamed: 16','最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量','漲跌(+/-)'], axis=1, inplace=True)
    for i in ['成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '本益比']:
        df[i] = df[i].apply(lambda x: pd.to_numeric(x.replace(",", ""), errors='coerce') if type(x)!=float else x)
    df['股價日期'] = datetime(int(year),int(month),int(day),0,0,0)
    return df

In [21]:
df = siiPrice(2020,7,28,False)

In [22]:
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,股價日期
0,0050,元大台灣50,22422761,11952,2379933200,107.25,108.70,103.30,103.60,0.30,0.0,2020-07-28
1,0051,元大中型100,175955,105,6843607,39.00,39.58,38.58,38.60,0.39,0.0,2020-07-28
2,0052,富邦科技,1109712,984,108400215,100.40,100.40,92.00,93.00,1.70,0.0,2020-07-28
3,0053,元大電子,278589,204,14469638,53.50,54.75,49.21,49.65,0.16,0.0,2020-07-28
4,0054,元大台商50,28217,14,714505,25.60,25.72,25.15,25.15,0.32,0.0,2020-07-28


In [8]:
year = 2020
month = 7
day = 28
url = f'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date={year}{month:02}{day:02}&type=ALL'

In [13]:
r = requests.get(url)
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)

In [14]:
df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,0050,元大台灣50,"22,422,761","11,952","2,379,933,200",107.25,108.70,103.30,103.60,+,0.30,103.55,3,103.60,23,0.00,NaN
1,0051,元大中型100,"175,955",105,"6,843,607",39.00,39.58,38.58,38.60,-,0.39,38.60,34,38.72,15,0.00,NaN
2,0052,富邦科技,"1,109,712",984,"108,400,215",100.40,100.40,92.00,93.00,+,1.70,93.00,2,93.05,1,0.00,NaN
3,0053,元大電子,"278,589",204,"14,469,638",53.50,54.75,49.21,49.65,-,0.16,49.65,2,49.80,1,0.00,NaN
4,0054,元大台商50,"28,217",14,"714,505",25.60,25.72,25.15,25.15,-,0.32,25.15,45,25.25,1,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17558,9944,新麗,"685,353",350,"13,286,536",20.00,20.00,19.10,19.25,+,0.10,19.20,6,19.30,12,0.00,NaN
17559,9945,潤泰新,"4,906,186","2,366","229,614,061",45.95,47.50,45.70,47.00,+,0.60,47.00,23,47.05,34,4.76,NaN
17560,9946,三發地產,"303,226",85,"4,677,340",15.60,15.60,15.30,15.35,-,0.25,15.35,9,15.40,5,3.84,NaN
17561,9955,佳龍,"1,528,155",785,"30,788,026",20.60,21.00,19.00,19.20,-,0.95,19.20,29,19.45,7,0.00,NaN


In [3]:
import sys
sys.path.insert(0, r'C:\Users\oo245\Documents\stocker')
from financial_statement import FS

In [5]:
fs = FS(fs_type='income', co_id='1101', latest=False, year=109, season=1, market='sii')

In [6]:
fs.clean()

True

In [7]:
fs.df

,income,amount
0,營業收入合計,21976200.00
1,營業成本合計,16195489.00
2,營業毛利（毛損）,5780711.00
3,營業毛利（毛損）淨額,5780711.00
4,營業費用,0.00
5,推銷費用,228564.00
6,管理費用,1088619.00
7,研究發展費用,3374.00
8,營業費用合計,1320557.00
9,營業利益（損失）,4460154.00


In [37]:
for df in fs.df:
    print(len(df))

1
1
1
1
2
1
1
1
0
1
3
59
0


In [11]:
fs.df[10].columns[1]

('民國104年第2季', '單位：新台幣仟元', '104年第2季', '金額')

In [7]:
url = 'https://mops.twse.com.tw/mops/web/t163sb06'           
form = {
    'encodeURIComponent':1,
    'step': 1,
    'firstin': 1,
    'off': 1,
    'TYPEK': 'sii',
    'year': '104',
    'season': '01'
}
        
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}    
r = requests.post(url, form, headers=headers)
r.encoding = 'utf8'
df = pd.read_html(StringIO(r.text))

In [8]:
form

{'encodeURIComponent': 1,
 'step': 1,
 'firstin': 1,
 'off': 1,
 'TYPEK': 'sii',
 'year': '104',
 'season': '01'}

In [5]:
fs.df

[                                                   0
 0  全站搜尋營收除權息電子書法說會庫藏股董監持股獨立董事董監酬金ETFTDRwindow.onl...,
     0                                 1
 0 NaN  營收除權息電子書法說會庫藏股董監持股獨立董事董監酬金ETFTDR,
                                                    0
 0  彙總報表  基本資料  股東會及股利  TDR股利分派情形(101年起適用)  除權息公告 ...,
                                                    0   1  \
 0  彙總報表  基本資料  股東會及股利  TDR股利分派情形(101年起適用)  除權息公告 ... NaN   
 
                                                    2  
 0  營益分析查詢彙總表  市場別  上市上櫃興櫃公開發行  年度  季別  1  2  3  4...  ,
                                                    0
 0                                               彙總報表
 1  基本資料  股東會及股利  TDR股利分派情形(101年起適用)  除權息公告  股東會及除...,
     0                                    1   2   3   4
 0 NaN  市場別  上市上櫃興櫃公開發行  年度  季別  1  2  3  4 NaN NaN NaN,
     0    1           2   3   4   5   6   7           8
 0 NaN  市場別  上市上櫃興櫃公開發行 NaN  年度 NaN NaN  季別  1  2  3  4,
     0   1
 0 NaN NaN,
 Empty DataFrame
 Columns: [Unnamed: 0, Unnamed: 1

In [4]:
fs.clean()

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [8]:
fs.get(step=2)

In [11]:
fs.df

,object,amount
0,營業活動之現金流量－間接法,0.0
1,繼續營業單位稅前淨利（淨損）,54110.0
2,本期稅前淨利（淨損）,54110.0
3,折舊費用,8023.0
4,攤銷費用,1247.0
5,呆帳費用提列（轉列收入）數,3486.0
6,利息收入,-14425.0
7,各項保險負債淨變動,839643.0
8,具金融商品性質之保險契約準備淨變動,0.0
9,其他各項負債準備淨變動,-2474.0


In [68]:
url = 'https://mops.twse.com.tw/mops/web/t164sb03'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'} 
form = {
    'encodeURIComponent': 1,
    'step': 2,
    'firstin': 1,
    'off': 1,
    'queryName': 'co_id',
    'inpuType': 'co_id',
    'TYPEK': 'all',
    'isnew': 'false',
    'co_id': str(2816),
    'year': str(104),
    'season': 4,
}
r = requests.post(url, form, headers=headers)
r.encoding = 'utf8'
df = pd.read_html(StringIO(r.text))

In [69]:
df[10]

民國104年第4季                                          \
                     單位：新台幣仟元                                           
                         會計項目  104年12月31日          103年12月31日           
           Unnamed: 0_level_3          金額       %          金額       %   
0                     現金及約當現金   2131322.0   14.73   2855073.0   20.66   
1                      應收款項淨額    767890.0    5.31    909928.0    6.58   
2                     本期所得稅資產     14070.0    0.10     10232.0    0.07   
3                          投資         NaN     NaN         NaN     NaN   
4            透過損益按公允價值衡量之金融資產    463077.0    3.20   1037345.0    7.51   
5                    備供出售金融資產    512119.0    3.54    612471.0    4.43   
6                  以成本衡量之金融資產      1320.0    0.01     13182.0    0.10   
7                  持有至到期日金融資產    247261.0    1.71    248222.0    1.80   
8                   其他金融資產－淨額   3803870.0   26.29   1950807.0   14.11   
9                    投資性不動產淨額    843212.0    5.83    859072.0    6.22   
10                       投資合計   5870859.0   40.57   4721099.0   34.16   
11                  再保險合約資產合計   3974927.0   27.47   3655979.0   26.45   
12                   不動產及設備合計    834018.0    5.76    811159.0    5.87   
13                     無形資產合計    101242.0    0.70    130814.0    0.95   
14                    遞延所得稅資產      3078.0    0.02         0.0    0.00   
15                     其他資產合計    773076.0    5.34    727444.0    5.26   
16                       資產總計  14470482.0  100.00  13821728.0  100.00   
17                     應付款項合計   1175467.0    8.12   1024563.0    7.41   
18                     保險負債小計   9408753.0   65.02   8907997.0   64.45   
19             具金融商品性質之保險契約準備     44087.0    0.30     74156.0    0.54   
20                       負債準備    208763.0    1.44    181793.0    1.32   
21                    遞延所得稅負債     64400.0    0.45     63920.0    0.46   
22                     其他負債合計     45832.0    0.32     69987.0    0.51   
23                       負債總計  10947302.0   75.65  10322416.0   74.68   
24                歸屬於母公司業主之權益         NaN     NaN         NaN     NaN   
25                         股本         NaN     NaN         NaN     NaN   
26                      普通股股本   2129600.0   14.72   2129600.0   15.41   
27                       股本合計   2129600.0   14.72   2129600.0   15.41   
28                     資本公積合計         0.0    0.00         0.0    0.00   
29                       保留盈餘         NaN     NaN         NaN     NaN   
30                     法定盈餘公積    243991.0    1.69    127589.0    0.92   
31                     特別盈餘公積   1050167.0    7.26    878518.0    6.36   
32                      未分配盈餘     69858.0    0.48    365435.0    2.64   
33                     保留盈餘合計   1364016.0    9.43   1371542.0    9.92   
34                       其他權益         NaN     NaN         NaN     NaN   
35          國外營運機構財務報表換算之兌換差額     -1028.0   -0.01         0.0    0.00   
36              備供出售金融資產未實現損益      1351.0    0.01     -1830.0   -0.01   
37                     其他權益合計       323.0    0.00     -1830.0   -0.01   
38                歸屬於母公司業主之權益   3493939.0   24.15   3499312.0   25.32   
39                      非控制權益     29241.0    0.20         0.0    0.00   
40                       權益總計   3523180.0   24.35   3499312.0   25.32   
41                    負債及權益總計  14470482.0  100.00  13821728.0  100.00   
42              待註銷股本股數（單位：股）         0.0     NaN         0.0     NaN   
43  母公司暨子公司所持有之母公司庫藏股股數（單位：股）         0.0     NaN         0.0     NaN   
44    預收股款（權益項下）之約當發行股數（單位：股）         0.0     NaN         0.0     NaN   

                                                              
                                                              
    103年01月01日         Unnamed: 7_level_2 Unnamed: 8_level_2  
            金額       % Unnamed: 7_level_3 Unnamed: 8_level_3  
0    1281581.0    9.04                NaN                NaN  
1     954305.0    6.73                NaN                NaN  
2      10971.0    0.08                NaN                NaN  
3

In [4]:
        self.season = season

for year in range(104, 110):
    for season in range(1,5):
        for fs_type in ['asset', 'income', 'cashflow', 'profit']:
            # try:
            if fs_type != 'profit':
                for co_id in index:
                    fname = f'C:\\Users\\oo245\\Documents\\stocker\\data\\{fs_type}_{year}_{season}_{co_id}.pkl'
                    if not os.path.exists(fname):
                        fs = FS(fs_type=fs_type, co_id=co_id, latest=False, year=year, season=season, market='sii')
                        fs.save()
                        rest = np.random.randint(3,8)
                        time.sleep(rest)
            else:
                fname = f'C:\\Users\\oo245\\Documents\\stocker\\data\\{fs_type}_{year}_{season}.pkl'
                if not os.path.exists(fname):
                    fs = FS(fs_type=fs_type, co_id='1232', latest=False, year=year, season=season, market='sii')
                    fs.save()
                    rest = np.random.randint(3,8)
                    time.sleep(rest)
            # except:
            #     print(f'{fs_type}_{year}_{season}')



NameError: name 'index' is not defined

In [17]:
fs.df[['index','name']].to_pickle('C:\\Users\\oo245\\Documents\\stocker\\data\\index.pkl')

In [9]:
df = pd.read_pickle(r'C:\Users\oo245\Documents\stocker\data\profit_104_2.pkl')

In [11]:
df[df['index']=='3598']

,index,name,毛利率,營業利益率,稅前純益率,稅後純益率
621,3598,奕力,19.13,3.85,3.63,3.16
